<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Politness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install anytree

In [6]:
!pip3 install convokit
!python3 -m spacy download en

2024-01-08 13:35:28.798201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 13:35:28.798256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 13:35:28.799516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 13:35:29.924903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.9 MB/s eta 0:00:00
✔ Download and installation successf

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
SAMPLE_SIZE = 3000

In [8]:
import pickle

# Reading from a file using Pickle
with open(f"{base_path}conversations_with_relatedness.pkl", 'rb') as file:
    conversations = pickle.load(file)

# Print the loaded variables
print("Len loaded data:", len(conversations))


Len loaded data: 32990


In [9]:
flat_comments_list = []
counter = 0
for id in conversations:
  comments = conversations[id]['comments']
  for comment in comments:
    flat_comments_list.append(comment)
  counter += 1
  if counter >= SAMPLE_SIZE:
    break

In [10]:
from convokit import Corpus, Utterance, Speaker
# Create a Corpus object from the flattened tree.
corpus = Corpus(utterances=[
        Utterance(id=node['id'], text=node['body'], speaker=Speaker(id = node['author']), reply_to=node['reply_to'], conversation_id=node['conversation_id'])
        for node in flat_comments_list
])

In [11]:
corpus.print_summary_stats()

Number of Speakers: 28803
Number of Utterances: 48922
Number of Conversations: 3000


In [12]:
from convokit.text_processing import TextParser
parser = TextParser(verbosity=1000)
corpus = parser.transform(corpus)

1000/48922 utterances processed
2000/48922 utterances processed
3000/48922 utterances processed
4000/48922 utterances processed
5000/48922 utterances processed
6000/48922 utterances processed
7000/48922 utterances processed
8000/48922 utterances processed
9000/48922 utterances processed
10000/48922 utterances processed
11000/48922 utterances processed
12000/48922 utterances processed
13000/48922 utterances processed
14000/48922 utterances processed
15000/48922 utterances processed
16000/48922 utterances processed
17000/48922 utterances processed
18000/48922 utterances processed
19000/48922 utterances processed
20000/48922 utterances processed
21000/48922 utterances processed
22000/48922 utterances processed
23000/48922 utterances processed
24000/48922 utterances processed
25000/48922 utterances processed
26000/48922 utterances processed
27000/48922 utterances processed
28000/48922 utterances processed
29000/48922 utterances processed
30000/48922 utterances processed
31000/48922 utteran

In [13]:
from convokit import politenessStrategies
ps = politenessStrategies.PolitenessStrategies(verbose = 1000)
corpus = ps.fit_transform(corpus=corpus)

1000/48922 utterances processed
2000/48922 utterances processed
3000/48922 utterances processed
4000/48922 utterances processed
5000/48922 utterances processed
6000/48922 utterances processed
7000/48922 utterances processed
8000/48922 utterances processed
9000/48922 utterances processed
10000/48922 utterances processed
11000/48922 utterances processed
12000/48922 utterances processed
13000/48922 utterances processed
14000/48922 utterances processed
15000/48922 utterances processed
16000/48922 utterances processed
17000/48922 utterances processed
18000/48922 utterances processed
19000/48922 utterances processed
20000/48922 utterances processed
21000/48922 utterances processed
22000/48922 utterances processed
23000/48922 utterances processed
24000/48922 utterances processed
25000/48922 utterances processed
26000/48922 utterances processed
27000/48922 utterances processed
28000/48922 utterances processed
29000/48922 utterances processed
30000/48922 utterances processed
31000/48922 utteran

In [14]:
ps.summarize(corpus=corpus)

feature_politeness_==Please==                 0.008810
feature_politeness_==Please_start==           0.004333
feature_politeness_==HASHEDGE==               0.249336
feature_politeness_==Indirect_(btw)==         0.000123
feature_politeness_==Hedges==                 0.113977
feature_politeness_==Factuality==             0.080087
feature_politeness_==Deference==              0.018928
feature_politeness_==Gratitude==              0.042026
feature_politeness_==Apologizing==            0.010302
feature_politeness_==1st_person_pl.==         0.078186
feature_politeness_==1st_person==             0.292098
feature_politeness_==1st_person_start==       0.263378
feature_politeness_==2nd_person==             0.254385
feature_politeness_==2nd_person_start==       0.058706
feature_politeness_==Indirect_(greeting)==    0.005928
feature_politeness_==Direct_question==        0.064715
feature_politeness_==Direct_start==           0.118290
feature_politeness_==HASPOSITIVE==            0.498181
feature_po

In [29]:
def calculate_politeness_score(politeness_features):
    # Number of features
    num_features = len(politeness_features)

    # Sum of feature values
    total_score = sum(politeness_features[feature] for feature in politeness_features)

    # Calculate the average (equal weights)
    politeness_score = total_score / num_features

    return politeness_score

In [32]:
from convokit import PolitenessStrategies

df = corpus.get_utterances_dataframe()
politness_score_dic = dict()
for index, row in df.iterrows():
    comment_id = index
    politeness_strategies = row['meta.politeness_strategies']
    politness_score_dic[comment_id] = calculate_politeness_score(politeness_strategies)


In [33]:
flat_comments_list = []
counter = 0
for id in conversations:
  comments = conversations[id]['comments']
  for comment in comments:
    comment['politness_score'] = politness_score_dic[comment['id']]
  counter += 1
  if counter >= SAMPLE_SIZE:
    break

In [35]:
import pickle

# Writing to a file using Pickle
with open(f"{base_path}conversations_with_politness.pkl", 'wb') as file:
    pickle.dump(conversations, file)

print("Variables written to the file.")

Variables written to the file.
